# Streptococcus comparative genomics

Comparing protein domain presence when a specific doamin such as LysM is present vs when it's not.

In [109]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from Bio import Phylo

cwd = os.getcwd()
if cwd.endswith('notebook'):
    os.chdir('..')
    cwd = os.getcwd()

from src.cell_wall_binding_domains import cwb_domains

In [110]:
sns.set_theme(palette='colorblind', font_scale=1.3)
palette_colorblind = sns.color_palette('colorblind').as_hex()
palette_pastel = sns.color_palette('pastel').as_hex()

data_folder = Path('./data/')
assert data_folder.is_dir()

db_proka = Path('../db_proka/')
assert db_proka.is_dir()

gtdb_folder = Path('../data/gtdb_r220/')
assert gtdb_folder.is_dir()

strep_folder = gtdb_folder / 'Streptococcus'
assert strep_folder.is_dir()

## Load data

In [111]:
metadata_df = pd.read_csv(strep_folder / 'genomes_metadata.csv', index_col='assembly_accession')
metadata_df.head()

,accession,ambiguous_bases,checkm2_completeness,checkm2_contamination,checkm2_model,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_900636555.1,RS_GCF_900636555.1,0,100.00,0.14,Specific,100.00,0.00,475,o__Lactobacillales (UID544),267,...,19,59,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus gordonii
GCF_015908985.1,RS_GCF_015908985.1,0,100.00,0.23,Specific,100.00,0.00,475,o__Lactobacillales (UID544),267,...,19,56,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus suis
GCF_001266635.1,RS_GCF_001266635.1,0,100.00,0.06,Specific,99.82,0.00,524,f__Streptococcaceae (UID545),282,...,18,63,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus agalactiae
GCF_004154025.1,RS_GCF_004154025.1,0,99.99,0.13,Specific,99.85,0.00,676,g__Streptococcus (UID722),182,...,19,67,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pyogenes
GCF_029011635.1,RS_GCF_029011635.1,0,100.00,0.20,Specific,100.00,0.18,524,f__Streptococcaceae (UID545),282,...,19,80,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus agalactiae


In [112]:
pfam_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.pfam.csv', index_col='assembly_accession')
pfam_df['gtdb_species'] = [metadata_df.loc[a, 'gtdb_species'] for a in pfam_df.index]

tigr_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.tigr.csv', index_col='assembly_accession')
tigr_df['gtdb_species'] = [metadata_df.loc[a, 'gtdb_species'] for a in tigr_df.index]

## How many LysM per genomes?

In [113]:
lysM_df = pfam_df[pfam_df['hmm_query'] == 'LysM'].copy()

n_copies_per_species = lysM_df.reset_index()[
    ['gtdb_species', 'id', 'assembly_accession']
].groupby('gtdb_species').nunique()

n_copies_per_species['ratio'] = n_copies_per_species['id'] / n_copies_per_species['assembly_accession']
n_copies_per_species.sort_values('assembly_accession', ascending=False).head(10)

,id,assembly_accession,ratio
gtdb_species,,,
Streptococcus pyogenes,271,271,1.000000
Streptococcus agalactiae,611,197,3.101523
Streptococcus pneumoniae,313,158,1.981013
Streptococcus suis,268,109,2.458716
Streptococcus thermophilus,310,88,3.522727
Streptococcus dysgalactiae,57,43,1.325581
Streptococcus mutans,68,26,2.615385
Streptococcus iniae,14,14,1.000000
Streptococcus suis_W,39,13,3.000000


In [114]:
s_pyogenes_repr = 'GCF_002055535.1'
df = pfam_df.loc[s_pyogenes_repr]
df[df['hmm_query'] == 'LysM']

,id,protein_id,hmm_accession,hmm_query,evalue,bitscore,accuracy,start,end,gtdb_species
assembly_accession,,,,,,,,,,
GCF_002055535.1,WP_010921965.1@GCF_002055535.1,WP_010921965.1,PF01476.25,LysM,1.600000e-10,37.6,0.97,48,91,Streptococcus pyogenes


## LysM in S. equi

We spotted a pattern of differential presence of LysM in certain strains of _Steptococcus equi_.

In [115]:
accessions_s_equi = metadata_df[
    metadata_df['gtdb_species'] == 'Streptococcus equi'
].index

pfam_s_equi = pfam_df.loc[accessions_s_equi]
tigr_s_equi = tigr_df.loc[accessions_s_equi]

accessions_yes_LysM = set(pfam_s_equi[
    pfam_s_equi['hmm_query'] == 'LysM'
].index)

accessions_no_LysM = set(accessions_s_equi) - accessions_yes_LysM

print(f'Number of S. equi genomes:              {len(accessions_s_equi):,}')
print(f'Number of S. equi genones with LysM:    {len(accessions_yes_LysM):,}')
print(f'Number of S. equi genones without LysM: {len(accessions_no_LysM):,}')

Number of S. equi genomes:              38
Number of S. equi genones with LysM:    4
Number of S. equi genones without LysM: 34


In [116]:
extra_in_no = (
    (
        set(pfam_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values)
    ) - 
    (
        set(pfam_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values)
    )
)
len(extra_in_no)

263

In [117]:
pfam_no_lysM = pfam_s_equi.loc[sorted(accessions_no_LysM)]
tigr_no_lysM = tigr_s_equi.loc[sorted(accessions_no_LysM)]

domain_with_count = []
for domain in extra_in_no:
    if 'TIGR' in domain:
        df = tigr_no_lysM
    else:
        df = pfam_no_lysM

    count = len(set(df[df['hmm_query'] == domain].index))
    percent = float(np.round(100 * count / len(accessions_no_LysM), 1))

    domain_with_count.append((domain, count, percent))

domain_with_count = sorted(domain_with_count, key=lambda t: t[1], reverse=True)
domain_with_count[:20]

[('TIGR03426', 26, 76.5),
 ('Malt_amylase_C', 24, 70.6),
 ('AAA_10', 21, 61.8),
 ('FokI_D3', 21, 61.8),
 ('FokI_D1', 21, 61.8),
 ('FokI_dom_2', 21, 61.8),
 ('FokI_cleav_dom', 21, 61.8),
 ('MethyltransfD12', 20, 58.8),
 ('Polysacc_synt_3', 20, 58.8),
 ('TIGR00571', 19, 55.9),
 ('Phage_int_SAM_5', 19, 55.9),
 ('Gp58', 18, 52.9),
 ('Cellulase', 17, 50.0),
 ('RloB', 16, 47.1),
 ('T6_Ig_like', 16, 47.1),
 ('HTH_36', 16, 47.1),
 ('TIGR01637', 15, 44.1),
 ('TIGR01451', 15, 44.1),
 ('DUF1617', 14, 41.2),
 ('GA', 14, 41.2)]

In [118]:
missing_in_no = (
    (
        set(pfam_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values)
    ) - 
    (
        set(pfam_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values)
    )
)
len(missing_in_no)

16

In [119]:
pfam_yes_lysM = pfam_s_equi.loc[sorted(accessions_yes_LysM)]
tigr_yes_lysM = tigr_s_equi.loc[sorted(accessions_yes_LysM)]

missing_domain_with_count = []
for domain in missing_in_no:
    if 'TIGR' in domain:
        df = tigr_yes_lysM
    else:
        df = pfam_yes_lysM

    count = len(set(df[df['hmm_query'] == domain].index))
    percent = float(np.round(100 * count / len(accessions_yes_LysM), 1))

    missing_domain_with_count.append((domain, count, percent))

missing_domain_with_count = sorted(missing_domain_with_count, key=lambda t: t[1], reverse=True)
missing_domain_with_count[:20]

[('DUF6900', 4, 100.0),
 ('Amidase_3', 4, 100.0),
 ('LysM', 4, 100.0),
 ('DUF2326', 3, 75.0),
 ('MC7', 3, 75.0),
 ('CTD11', 3, 75.0),
 ('PhdYeFM_antitox', 3, 75.0),
 ('ToxN_toxin', 3, 75.0),
 ('SLFN-g3_helicase', 3, 75.0),
 ('MeaB', 3, 75.0),
 ('DEDD_Tnp_IS110', 2, 50.0),
 ('HNH_3', 1, 25.0),
 ('Imm63', 1, 25.0),
 ('DUF4145', 1, 25.0),
 ('DUF3781', 1, 25.0),
 ('TrbL', 1, 25.0)]

Not particularly conclusive at this level of analysis.

## Cell wall HMM search

HMM models from [Megrian et al., 2022](https://doi.org/10.1038/s41564-022-01257-y).

In [120]:
cw_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.cell_wall.csv', index_col='assembly_accession')

# Keep only one model per protein – best scoring one.
cw_df = cw_df.sort_values(['id', 'bitscore'], ascending=[True, False]).drop_duplicates(['id'])

cw_df.head()

,id,protein_id,hmm_accession,hmm_query,evalue,bitscore,accuracy,start,end
assembly_accession,,,,,,,,,
GCA_000013525.1,ABF35082.1@GCA_000013525.1,ABF35082.1,newDdlB,newDdlB,6.500000e-08,28.6,0.77,126,303
GCA_000013525.1,ABF35084.1@GCA_000013525.1,ABF35084.1,newDdlB,newDdlB,3.500000e-11,39.3,0.79,106,289
GCA_000013525.1,ABF35137.1@GCA_000013525.1,ABF35137.1,newFtsI,newFtsI,1.900000e-21,73.0,0.79,329,642
GCA_000013525.1,ABF35311.1@GCA_000013525.1,ABF35311.1,newMraY,newMraY,1.400000e-35,119.7,0.79,0,330
GCA_000013525.1,ABF35345.1@GCA_000013525.1,ABF35345.1,newMurD,newMurD,3.800000e-05,19.8,0.77,0,117


In [121]:
cw_df_yes_lysM = cw_df.loc[sorted(accessions_yes_LysM)]

cw_yes_lysM_grouped = cw_df_yes_lysM[
    ['id', 'hmm_query']
].groupby(
    'hmm_query'
).nunique().sort_values('id', ascending=False).rename(
    columns={
        'id': 'count',
    }
)
cw_yes_lysM_grouped['ratio'] = cw_yes_lysM_grouped['count'] / len(accessions_yes_LysM)
cw_yes_lysM_grouped

,count,ratio
hmm_query,,
newDdlB,20,5.0
newFtsI,20,5.0
newMraW,12,3.0
newMurA,12,3.0
newMurD,12,3.0
newFtsA,8,2.0
newFtsW,8,2.0
newMraY,8,2.0
newMurE,8,2.0


In [122]:
cw_df_no_lysM = cw_df.loc[sorted(accessions_no_LysM)]

cw_no_lysM_grouped = cw_df_no_lysM[
    ['id', 'hmm_query']
].groupby(
    'hmm_query'
).nunique().sort_values('id', ascending=False).rename(
    columns={
        'id': 'count',
    }
)
cw_no_lysM_grouped['ratio'] = cw_no_lysM_grouped['count'] / len(accessions_no_LysM)
cw_no_lysM_grouped

,count,ratio
hmm_query,,
newFtsI,170,5.000000
newDdlB,169,4.970588
newMurA,102,3.000000
newMurD,101,2.970588
newMraW,94,2.764706
newFtsW,68,2.000000
newMraY,68,2.000000
newFtsA,67,1.970588
newMurC,52,1.529412


In [123]:
set(cw_no_lysM_grouped.index) - set(cw_yes_lysM_grouped.index)

{'newMraZ'}

In [124]:
set(cw_yes_lysM_grouped.index) - set(cw_no_lysM_grouped.index)

set()

## eggNOG orthologs

In [102]:
eggNOG_annotations_path = strep_folder / 'Streptococcus_eggNOG_annotations.csv'

if not eggNOG_annotations_path.is_file():
    eggNOG_df = pd.read_csv(
        strep_folder / 'eggNOG' / 'Streptococcus.emapper.annotations',
        sep='\t',
        comment='#',
        header=None,
        names=[
            'protein_id', 'seed_ortholog', 'evalue', 'score', 'eggNOG_OGs', 'max_annot_lvl', 
            'COG_category', 'Description', 'Preferred_name', 'GOs', 'EC', 
            'KEGG_ko', 'KEGG_Pathway', 'KEGG_Module', 'KEGG_Reaction', 'KEGG_rclass', 
            'BRITE', 'KEGG_TC', 'CAZy', 'BiGG_Reaction', 'PFAMs',
        ]
    )

    eggNOG_df['assembly_accession'] = eggNOG_df['protein_id'].apply(lambda v: v.split('@')[1])
    eggNOG_df = eggNOG_df.set_index('assembly_accession', drop=True).replace('-', np.nan)

    eggNOG_df.to_csv(eggNOG_annotations_path)

eggNOG_df = pd.read_csv(eggNOG_annotations_path, index_col='assembly_accession')
eggNOG_df.head()

,protein_id,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_003963555.1,WP_126467658.1@GCF_003963555.1,1000570.HMPREF9966_1759,9.704000e-82,271.0,"COG0716@1|root,COG0716@2|Bacteria,1V45R@1239|F...",2|Bacteria,C,Flavodoxin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flavodoxin_4
GCF_003963555.1,WP_126467724.1@GCF_003963555.1,862970.SAIN_1577,1.313000e-186,582.0,"COG1396@1|root,COG1396@2|Bacteria,1VIH9@1239|F...",2|Bacteria,K,Helix-turn-helix XRE-family like proteins,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"HTH_19,HTH_3,TPR_12,TPR_8"
GCF_003963555.1,WP_126467790.1@GCF_003963555.1,862970.SAIN_1636,2.776000e-131,418.0,"COG1564@1|root,COG1564@2|Bacteria,1VA0W@1239|F...",2|Bacteria,H,"Thiamin pyrophosphokinase, vitamin B1 binding ...",thiN,NaN,...,ko:K00949,"ko00730,ko01100,map00730,map01100",NaN,R00619,"RC00002,RC00017","ko00000,ko00001,ko01000",NaN,NaN,NaN,"TPK_B1_binding,TPK_catalytic"
GCF_003963555.1,WP_126467857.1@GCF_003963555.1,176090.SSIN_0693,1.181000e-201,627.0,"COG3677@1|root,COG3677@2|Bacteria,1V4D1@1239|F...",2|Bacteria,L,ISXO2-like transposase domain,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"DDE_Tnp_IS1595,Zn_Tnp_IS1595"
GCF_003963555.1,WP_126467993.1@GCF_003963555.1,862969.SCI_1925,1.812000e-256,791.0,"COG0612@1|root,COG0612@2|Bacteria,1TPN6@1239|F...",2|Bacteria,S,Peptidase M16 inactive,ymfF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Peptidase_M16,Peptidase_M16_C"


In [133]:
s_equi_yes_lysM_instance = sorted(accessions_yes_LysM)[0]
metadata_df.loc[s_equi_yes_lysM_instance, 'ncbi_organism_name']

'Streptococcus equi subsp. zooepidemicus ATCC 35246'

In [140]:
df = pfam_df.loc[s_equi_yes_lysM_instance]
s_equi_yes_lysM_protein_id = df[df['hmm_query'] == 'LysM']['id'].values[0]
s_equi_yes_lysM_protein_id

'WP_014622634.1@GCF_000219765.1'

In [161]:
r = eggNOG_df[eggNOG_df['protein_id'] == s_equi_yes_lysM_protein_id].iloc[0]

print(r['eggNOG_OGs'])

r

COG0860@1|root,COG1388@1|root,COG0860@2|Bacteria,COG1388@2|Bacteria,1VINV@1239|Firmicutes,4HPIN@91061|Bacilli


protein_id                           WP_014622634.1@GCF_000219765.1
seed_ortholog                          1123299.AUKZ01000026_gene846
evalue                                                          0.0
score                                                         786.0
eggNOG_OGs        COG0860@1|root,COG1388@1|root,COG0860@2|Bacter...
max_annot_lvl                                            2|Bacteria
COG_category                                                      M
Description                      N-acetylmuramoyl-L-alanine amidase
Preferred_name                                                 yocH
GOs                                                             NaN
EC                                         3.2.1.1,3.5.1.28,3.5.2.6
KEGG_ko           ko:K01176,ko:K01448,ko:K02067,ko:K06385,ko:K17...
KEGG_Pathway      ko00311,ko00500,ko01100,ko01130,ko01501,ko0150...
KEGG_Module               M00210,M00627,M00628,M00669,M00670,M00727
KEGG_Reaction                    R02108,R02112,R

In [163]:
s_iniae_acc = 'GCF_030732225.1'
df = pfam_df.loc[s_iniae_acc]
s_iniae_yes_lysM_protein_id = df[df['hmm_query'] == 'LysM']['id'].values[0]

eggNOG_df[eggNOG_df['protein_id'] == s_iniae_yes_lysM_protein_id].iloc[0]

r2 = eggNOG_df[eggNOG_df['protein_id'] == s_iniae_yes_lysM_protein_id].iloc[0]

print(r2['eggNOG_OGs'])

r2

COG1388@1|root,COG1388@2|Bacteria,1V61Q@1239|Firmicutes,4HHT0@91061|Bacilli


protein_id                           WP_016355818.1@GCF_030732225.1
seed_ortholog                                       1346.DQ08_01735
evalue                                                          0.0
score                                                         705.0
eggNOG_OGs        COG1388@1|root,COG1388@2|Bacteria,1V61Q@1239|F...
max_annot_lvl                                            2|Bacteria
COG_category                                                      M
Description                                     LysM domain protein
Preferred_name                                                  sip
GOs                                                             NaN
EC                                                              NaN
KEGG_ko                                                         NaN
KEGG_Pathway                                                    NaN
KEGG_Module                                                     NaN
KEGG_Reaction                                   

In [156]:
accessions_with_S_protein = sorted(set(eggNOG_df[eggNOG_df['eggNOG_OGs'].str.contains('4HHT0')].index))
strep_with_S_protein = metadata_df.loc[accessions_with_S_protein]

In [159]:
strep_with_S_protein[strep_with_S_protein['gtdb_species'] == 'Streptococcus equi']

,accession,ambiguous_bases,checkm2_completeness,checkm2_contamination,checkm2_model,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species
assembly_accession,,,,,,,,,,,,,,,,,,,,,


In [232]:
og_metadata = pd.read_csv(
    strep_folder / 'eggNOG' / 'e5.og_annotations.tsv',
    sep='\t', 
    header=None,
    names=['og', 'og_level', 'description'],
).drop_duplicates('og').set_index('og')
og_metadata.head()

,og_level,description
og,,
4HBBX,L,Recombinase
1XVC3,K,transcriptional regulator
2TDXG,S,NaN
2QWWR,S,NaN
37HD9,S,NaN


In [233]:
def display_ogs(ogs):
    for og in ogs:
        desc = og_metadata.loc[og, 'description']
        print(f'{og}: {desc}')

In [370]:
def get_unique_ogs(eggNOG_df, accessions, og_whitelist=None):
    index = eggNOG_df.index

    og_union = set()
    for acc in accessions:
        if acc not in index:
            print(f'Not in index: {acc}')
            continue

        genme_ogs = eggNOG_df.loc[[acc]].set_index('protein_id')

        ogs = set()
        for protein_id in genme_ogs.index:
            protein_ogs = genme_ogs.loc[protein_id, 'eggNOG_OGs']

            candidate_ogs = {
                og_with_tax.split('@')[0]
                for og_with_tax in protein_ogs.split(',')
            }

            if og_whitelist is not None:
                candidate_ogs = candidate_ogs & og_whitelist
                
            ogs |= candidate_ogs

        og_union = og_union.union(ogs)

    return og_union

In [371]:
def get_ogs_present_in_all(eggNOG_df, accessions, og_whitelist=None):
    index = eggNOG_df.index

    og_intersection = None
    for acc in accessions:
        if acc not in index:
            print(f'Not in index: {acc}')
            continue

        genme_ogs = eggNOG_df.loc[[acc]].set_index('protein_id')

        ogs = set()
        for protein_id in genme_ogs.index:
            protein_ogs = genme_ogs.loc[protein_id, 'eggNOG_OGs']

            candidate_ogs = {
                og_with_tax.split('@')[0]
                for og_with_tax in protein_ogs.split(',')
            }

            if og_whitelist is not None:
                candidate_ogs = candidate_ogs & og_whitelist
                
            ogs |= candidate_ogs

        if og_intersection is None:
            og_intersection = ogs
        else:
            og_intersection &= ogs

    return og_intersection

### S-protein

 (OG: `4HHT0`)

LysM-containing protein that is essential for immune evasion in Group A Streptococci. 

Refs: 
- [Wierzbicki et al., 2019](https://doi.org/10.1016/j.celrep.2019.11.001)
- [Burnier et al., 2024](https://doi.org/10.1101/2024.11.08.622053)

This S-protein isn't present in _S. equi_ (group C Strep). Some _S. equi_ strains do however contain OG `4HPIN` (N-acetylmuramoyl-L-alanine amidase) along with a LysM domain.

Intriguing... I wonder if the strains of _S. equi_ with a PGH have re-acquired a LysM-containing protein for the purpose of targeting other Streptococci?

Below is a list of the unique OGs of proteins with LysM in Streptococcus.

In [315]:
genomes_with_lysM = sorted(set(pfam_df[pfam_df['hmm_query'] == 'LysM'].index))
protein_ids_with_lysM = sorted(pfam_df[pfam_df['hmm_query'] == 'LysM']['id'].unique())

ogs_list_str = eggNOG_df[eggNOG_df['protein_id'].isin(protein_ids_with_lysM)]['eggNOG_OGs'].unique()
unique_ogs = sorted({
    og_with_taxa.split('@')[0]
    for og_list in ogs_list_str
    for og_with_taxa in og_list.split(',')
})
len(unique_ogs)

88

In [307]:
unique_cogs = [og for og in unique_ogs if og.startswith('COG')]
display_ogs(unique_cogs)

COG0741: lytic transglycosylase activity
COG0860: N-acetylmuramoyl-L-alanine amidase activity
COG1388: cell wall organization
COG1652: LysM domain
COG1705: Flagellar rod assembly protein muramidase FlgJ
COG2304: hyaluronan metabolic process
COG2340: Cysteine-rich secretory protein family
COG3170: Tfp pilus assembly protein FimV
COG3583: domain protein
COG3757: hydrolase, family 25
COG3942: CHAP domain
COG5632: N-acetylmuramoyl-L-alanine amidase activity


In [321]:
acc = metadata_df[
    (metadata_df['gtdb_species'] == 'Streptococcus pyogenes') &
    (metadata_df['gtdb_representative'] == 't')
].index[0]
df = pfam_df.loc[acc]
protein_ids = df[df['hmm_query'] == 'LysM']['id'].unique()
assert len(protein_ids) == 1
protein_id = protein_ids[0]

print(protein_id)
eggNOG_df[eggNOG_df['protein_id'] == protein_id]['eggNOG_OGs'].iloc[0]

WP_010921965.1@GCF_002055535.1


'COG1388@1|root,COG1388@2|Bacteria,1V61Q@1239|Firmicutes,4HHT0@91061|Bacilli'

In [329]:
accs = sorted(metadata_df[
    metadata_df['gtdb_species'].isin([
        'Streptococcus pyogenes',
        'Streptococcus dysgalactiae',
    ])
].index)
df = pfam_df.loc[accs]
protein_ids_with_lysM = sorted(df[df['hmm_query'] == 'LysM']['id'].unique())

eggNOG_df_subset = eggNOG_df[eggNOG_df['protein_id'].isin(protein_ids_with_lysM)].copy()

unique_ogs = get_ogs_present_in_all(eggNOG_df_subset, accs)

display_ogs(unique_ogs)

1V61Q: LysM domain protein
COG1388: cell wall organization
4HHT0: LysM domain protein


In [353]:
accessions_with_4HHT0 = sorted(set(eggNOG_df[eggNOG_df['eggNOG_OGs'].str.contains('4HHT0@')].index))

species_with_4HHT0 = pd.DataFrame(metadata_df.loc[accessions_with_4HHT0]['gtdb_species'].value_counts(), columns=['count'])
species_with_4HHT0['total'] = [len(metadata_df[metadata_df['gtdb_species'] == s]) for s in species_with_4HHT0.index]
species_with_4HHT0['percent'] = (100 * species_with_4HHT0['count'] / species_with_4HHT0['total']).round(0)
species_with_4HHT0.head(20)

,count,total,percent
gtdb_species,,,
Streptococcus pyogenes,271,271,100.0
Streptococcus agalactiae,196,197,99.0
Streptococcus pneumoniae,156,158,99.0
Streptococcus suis,108,109,99.0
Streptococcus dysgalactiae,43,43,100.0
Streptococcus iniae,13,14,93.0
Streptococcus suis_W,13,13,100.0
Streptococcus gordonii,13,13,100.0
Streptococcus salivarius,11,11,100.0


Let's see what's different between _S. equi_ strains without LysM and other strains with this S-protein. We'll exclude the S. equi strains with LysM for now.

In [216]:
ogs_no_lysM = get_ogs_present_in_all(eggNOG_df, sorted(accessions_no_LysM))
len(ogs_no_lysM)

4092

In [220]:
accessions_s_iniae_s_dysgalactiae = sorted(set(metadata_df[
    (metadata_df['gtdb_species'] == 'Streptococcus iniae') |
    (metadata_df['gtdb_species'] == 'Streptococcus dysgalactiae')
].index))

# og_yes_lysM = get_ogs_present_in_all(eggNOG_df, accessions_s_iniae_s_dysgalactiae)
og_yes_lysM = get_ogs_present_in_all(eggNOG_df, accessions_yes_LysM)
len(og_yes_lysM)

5660

In [238]:
missing_in_yes_lysM = ogs_no_lysM - og_yes_lysM
print('Gain:', len(ogs_no_lysM - og_yes_lysM), 'OGs')
display_ogs(sorted(missing_in_yes_lysM))

Gain: 28 OGs
1M8ZB: ABC-type amino acid transport signal transduction systems, periplasmic component domain
1M8ZC: Membrane protein involved in the export of O-antigen and teichoic acid
1M91B: Flavin transferase that catalyzes the transfer of the FMN moiety of FAD and its covalent binding to the hydroxyl group of a threonine residue in a target flavoprotein
1M96V: Periplasmic binding protein
1M9GP: leucine-rich protein
1M9KM: Specifically methylates the pseudouridine at position 1915 (m3Psi1915) in 23S rRNA
1MA2F: Nicotinamide mononucleotide transporter
1TQUG: ABC transporter
1TR9C: Flavin transferase that catalyzes the transfer of the FMN moiety of FAD and its covalent binding to the hydroxyl group of a threonine residue in a target flavoprotein
1UIJ9: ABC transporter
1V21G: nicotinamide mononucleotide transporter
1V3JM: Specifically methylates the pseudouridine at position 1915 (m3Psi1915) in 23S rRNA
1V649: COG2508, regulator of polyketide synthase expression
1W2RT: nan
2DEGR: nan
2

In [222]:
missing_in_no_lysM = og_yes_lysM - ogs_no_lysM
print(len(missing_in_no_lysM))

1596


In [243]:
m = metadata_df.loc[sorted(accessions_no_LysM)]
acc = m[m['gtdb_representative'] == 't'].index[0]

df = eggNOG_df.loc[acc]
df[df['eggNOG_OGs'].str.contains('1M8ZC')]

,protein_id,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCA_900637675.1,VEH28842.1@GCA_900637675.1,40041.SZO_00080,6.156000e-301,925.0,"COG2244@1|root,COG2244@2|Bacteria,1TNYX@1239|F...",2|Bacteria,S,Membrane protein involved in the export of O-a...,yabM,"GO:0005575,GO:0005623,GO:0005886,GO:0016020,GO...",...,"ko:K03328,ko:K06409",NaN,NaN,NaN,NaN,"ko00000,ko02000","2.A.66.2,2.A.66.2.14",NaN,NaN,"Polysacc_synt,Polysacc_synt_C"


In [247]:
df = eggNOG_df[eggNOG_df['eggNOG_OGs'].str.contains('1M8ZC')]

metadata_df.loc[sorted(set(df.index))]['gtdb_species'].value_counts()

gtdb_species
Streptococcus dysgalactiae    43
Streptococcus equi            37
Streptococcus halichoeri       1
Name: count, dtype: int64

## Focus on cell wall related pathways

In [340]:
kegg_pathways = [
    'map00550',  # Peptidoglycan biosynthesis
    'map00540',  # Lipopolysaccharide biosynthesis
]

COGs_only = True  # Focus on OGs with a COG ID (no internal eggNOG ones)

cw_ogs = set()
for kegg_pathway in kegg_pathways:
    ogs_str = eggNOG_df[
        eggNOG_df['KEGG_Pathway'].notnull() &
        eggNOG_df['KEGG_Pathway'].str.contains(kegg_pathway)
    ]['eggNOG_OGs'].values

    if COGs_only:
        cw_ogs |= {
            og
            for og_str in ogs_str
            for og_raw in og_str.split(',')
            if (og := og_raw.split('@')[0]).startswith('COG')
        }
    else:
        cw_ogs |= {
            og_raw.split('@')[0]
            for og_str in ogs_str
            for og_raw in og_str.split(',')
        }

len(cw_ogs)

44

In [372]:
ogs_no_lysM = get_ogs_present_in_all(eggNOG_df, sorted(accessions_no_LysM), cw_ogs)
ogs_no_lysM_all = get_unique_ogs(eggNOG_df, sorted(accessions_no_LysM), cw_ogs)
len(ogs_no_lysM), len(ogs_no_lysM_all)

(31, 36)

In [373]:
accessions = sorted(set(metadata_df[
    (metadata_df['gtdb_species'] == 'Streptococcus iniae') |
    (metadata_df['gtdb_species'] == 'Streptococcus pyogens') |
    (metadata_df['gtdb_species'] == 'Streptococcus dysgalactiae')
].index))

df = eggNOG_df.loc[accessions]
accessions = sorted(set(df[df['eggNOG_OGs'].str.contains('4HHT0')].index))

og_yes_lysM = get_ogs_present_in_all(eggNOG_df, accessions, cw_ogs)
og_yes_lysM_all = get_unique_ogs(eggNOG_df, accessions, cw_ogs)
len(og_yes_lysM), len(og_yes_lysM_all)

(23, 39)

In [365]:
missing_in_yes_lysM = ogs_no_lysM - og_yes_lysM
print('Gain:', len(missing_in_yes_lysM), 'OGs')
display_ogs(sorted(missing_in_yes_lysM))

Gain: 9 OGs
COG0770: UDP-N-acetylmuramoyl-tripeptide-D-alanyl-D-alanine ligase activity
COG0810: Interacts with outer membrane receptor proteins that carry out high-affinity binding and energy dependent uptake into the periplasmic space of specific substrates. It could act to transduce energy from the cytoplasmic membrane to specific energy- requiring processes in the outer membrane, resulting in the release into the periplasm of ligands bound by these outer membrane proteins
COG0818: Recycling of diacylglycerol produced during the turnover of membrane phospholipid
COG1181: D-alanine-D-alanine ligase activity
COG1680: COG1680 Beta-lactamase class C and other penicillin binding
COG3064: Membrane
COG3087: peptidoglycan binding
COG3266: translation initiation factor activity
COG4932: pathogenesis


In [356]:
m = metadata_df.loc[sorted(accessions_no_LysM)]
acc = m[m['gtdb_representative'] == 't'].index[0]
df = eggNOG_df.loc[acc]

protein_ids = set()
for og in sorted(missing_in_yes_lysM):
    protein_ids |= set(df[df['eggNOG_OGs'].str.contains(og)]['protein_id'].unique())

eggNOG_df[eggNOG_df['protein_id'].isin(sorted(protein_ids))]

,protein_id,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCA_900637675.1,VEH30782.1@GCA_900637675.1,40041.SZO_14790,9.833000e-27,115.0,"COG0810@1|root,COG0810@2|Bacteria,1W3PZ@1239|F...",2|Bacteria,M,Ortholog of S. equi 4047 (FM204883),NaN,NaN,...,"ko:K03646,ko:K03832",NaN,NaN,NaN,NaN,"ko00000,ko02000","2.C.1.1,2.C.1.2",NaN,NaN,"Gram_pos_anchor,TonB_C"
GCA_900637675.1,VEH30305.1@GCA_900637675.1,40041.SZO_16370,1.364000e-92,316.0,"COG0810@1|root,COG0810@2|Bacteria,1W3PZ@1239|F...",2|Bacteria,M,Ortholog of S. equi 4047 (FM204883),NaN,NaN,...,"ko:K03646,ko:K03832",NaN,NaN,NaN,NaN,"ko00000,ko02000","2.C.1.1,2.C.1.2",NaN,NaN,"Gram_pos_anchor,TonB_C"
GCA_900637675.1,VEH35905.1@GCA_900637675.1,1123299.AUKZ01000018_gene178,8.178000e-39,146.0,"COG3266@1|root,COG3266@2|Bacteria",2|Bacteria,GM,"domain, Protein",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Collagen,DUF285,FG-GAP_2,Gram_pos_anchor,YSIRK..."
GCA_900637675.1,VEH36050.1@GCA_900637675.1,40041.SZO_18110,2.772000e-89,302.0,"COG3064@1|root,COG3064@2|Bacteria,1UI3E@1239|F...",2|Bacteria,M,Collagen triple helix repeat (20 copies),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Collagen,Phage_fiber_C,VPEP,phage_tail_N"
GCA_900637675.1,VEH29168.1@GCA_900637675.1,40041.SZO_03720,7.049000e-120,396.0,"COG3266@1|root,COG3266@2|Bacteria,1VG02@1239|F...",2|Bacteria,M,Collagen triple helix repeat (20 copies),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Collagen,DUF2479,Gram_pos_anchor"
GCA_900637675.1,VEH35040.1@GCA_900637675.1,482234.SCAZ3_03785,1.987000e-32,134.0,"COG3064@1|root,COG3064@2|Bacteria",2|Bacteria,M,translation initiation factor activity,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Collagen,Phage_fiber_C,VPEP,phage_tail_N"
GCA_900637675.1,VEH30746.1@GCA_900637675.1,40041.SZO_05350,3.712000e-151,487.0,"COG3266@1|root,COG3266@2|Bacteria,1UI38@1239|F...",2|Bacteria,M,Pfam:Fb_signal,NaN,"GO:0005575,GO:0005623,GO:0030115,GO:0030312,GO...",...,"ko:K12132,ko:K21471",NaN,NaN,NaN,NaN,"ko00000,ko01000,ko01001,ko01002,ko01011",NaN,NaN,NaN,NaN
GCA_900637675.1,VEH35194.1@GCA_900637675.1,40041.SZO_03790,5.628000e-74,249.0,"COG3064@1|root,COG3064@2|Bacteria,1VP8T@1239|F...",2|Bacteria,M,Membrane,NaN,NaN,...,"ko:K08307,ko:K19140",NaN,NaN,NaN,NaN,"ko00000,ko01000,ko01011,ko02048",NaN,NaN,NaN,"DDE_Tnp_1,DUF772"
GCA_900637675.1,VEH32105.1@GCA_900637675.1,40041.SZO_11530,5.615000e-146,470.0,"COG0810@1|root,COG0810@2|Bacteria,1W3PZ@1239|F...",2|Bacteria,M,Ortholog of S. equi 4047 (FM204883),NaN,NaN,...,"ko:K03646,ko:K03832",NaN,NaN,NaN,NaN,"ko00000,ko02000","2.C.1.1,2.C.1.2",NaN,NaN,"Gram_pos_anchor,TonB_C"


In [357]:
missing_in_no_lysM = og_yes_lysM - ogs_no_lysM
print('Loss:', len(missing_in_no_lysM), 'OGs')
display_ogs(sorted(missing_in_no_lysM))

Loss: 1 OGs
COG2815: protein serine/threonine kinase activity


In [379]:
display_ogs(sorted(og_yes_lysM_all - ogs_no_lysM_all))

COG0451: coenzyme binding
COG1442: glycosyl transferase family 8
COG3147: Non-essential cell division protein that could be required for efficient cell constriction


In [381]:
acc = metadata_df[(metadata_df['gtdb_species'] == 'Streptococcus pyogenes') & (metadata_df['gtdb_representative'] == 't')].index[0]
df = eggNOG_df.loc[acc]

protein_ids = set()
for og in sorted(missing_in_no_lysM):
    protein_ids |= set(df[df['eggNOG_OGs'].str.contains(og)]['protein_id'].unique())

eggNOG_df[eggNOG_df['protein_id'].isin(sorted(protein_ids))]

,protein_id,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_002055535.1,WP_010922538.1@GCF_002055535.1,1314.HKU360_01383,0.0,1197.0,"COG0515@1|root,COG2815@1|root,COG0515@2|Bacter...",2|Bacteria,KLT,serine threonine protein kinase,prkC,"GO:0002237,GO:0003674,GO:0003824,GO:0004672,GO...",...,"ko:K08884,ko:K12132",NaN,NaN,NaN,NaN,"ko00000,ko01000,ko01001",NaN,NaN,NaN,"PASTA,Pkinase"
